In [23]:
%pip install plotly scikit-learn numpy pandas nbformat pyhull

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.8/318.8 kB 7.2 MB/s eta 0:00:0000:01
  Preparing metadata (setup.py) ... done
  Created wheel for pyhull: filename=pyhull-2015.2.1-cp312-cp312-macosx_11_0_arm64.whl size=174840 sha256=dfcb04090c9461ddf4333a2e1f7355d5625aa23de9a201d9c03e1e164a61e242
  Stored in directory: /Users/rorywaite/Library/Caches/pip/wheels/f4/f5/3a/b61e2fa2b618f779bbdb9e71a9f7e6916dbc9f05c3bab358c4
Successfully built pyhull
Note: you may need to restart the kernel to use updated packages.


In [31]:
from sklearn.datasets import make_blobs
import numpy as np
import json
from collections import defaultdict, namedtuple
import plotly.express as px
from pyhull.convex_hull import ConvexHull
from itertools import product

In [3]:
rs = np.random.RandomState(42)
num_cluster = 10
num_dim = 3

centres = [[rs.rand() for _ in range(num_dim)] for _ in range(num_cluster)]
sigmas = [rs.rand() for _ in range(10)]
blobs = make_blobs(
                   n_samples=100, 
                   n_features=num_dim,
                   centers=centres,
                   cluster_std=sigmas,
                   random_state=rs
                   ) 

In [6]:
point_clouds = [[] for _ in range(num_cluster)]
for sample, cluster in zip(blobs[0], blobs[1]):
    point_clouds[int(cluster)].append(sample.tolist())
inputs = [{"vertices" : points} for points in point_clouds]
with open("synthetic.json", "wt") as out_file:
    json.dump(inputs, out_file, indent=2)

In [7]:
with open("synthetic_out.json", "rt") as in_file:
    polys = json.load(in_file)

In [8]:
print(len(polys[5]["essential_indices"]))

8


In [9]:
poly_ind = 0
vertices = np.array(polys[poly_ind]["vertices"])
constant = np.ones([len(vertices), 1], dtype=float)
vertices = np.concatenate([constant, vertices], axis=1)
certs = {extreme: np.array(cert) for extreme, cert in polys[poly_ind]["essential_certs"].items()}

for vertex, cert in certs.items():
    scores = vertices @ cert
    print(scores, vertex, np.argmin(scores))

[-2.08959101e+00  3.63087338e-12 -1.63927999e+00 -2.65557562e+01
  2.54285482e-12 -7.55564118e+00 -3.76643191e+01  1.00000000e+00
 -1.26316542e+01  1.00008890e-12] 7 6
[-7.29431196e-01  1.15973897e-12 -2.75665424e-01  2.56411559e-12
  2.64444022e-12 -5.81603188e-02  1.00000000e+00 -2.63284624e-01
 -2.47089302e-01 -8.53040640e-01] 6 9
[-2.30012395e+00 -1.78923630e+00  1.00000000e+00 -4.04672882e-02
  1.85385041e-12 -4.56830358e-01  2.46469511e-12 -5.89222875e-01
  1.24700250e-12 -5.48354881e-01] 2 0
[ 1.07772125e-12  1.00000000e+00 -1.10466153e+00 -1.85085354e+00
  1.89187555e-12 -3.68464584e-01 -1.09688472e+00 -1.09336382e-02
 -1.27667341e+00 -1.02450723e+00] 1 3
[-3.69357486e+00  2.59436916e-12  3.15791837e-12 -7.49136587e+00
  1.00000000e+00 -1.88345206e+00 -7.25269019e+00  2.61857203e-12
 -3.95700769e+00 -2.51273734e+00] 4 3
[ 1.00000000e+00  1.15962795e-12 -1.33384026e+00  2.48795429e-12
 -1.04660325e+00 -4.38850256e-01 -4.46660383e-01 -5.16433924e-01
 -3.49783275e-01  1.97919459e-

In [10]:
poly_ind = 6
poly = polys[poly_ind]

dims = ["x", "y", "z"]
vertices_by_col = {dims[i]: list(col) for i, col in enumerate(zip(*poly["vertices"]))}
vertices_by_col["extreme"] = [str(i in poly["essential_indices"]) for i in range(len(poly["vertices"]))]
vertices_by_col["text"] = [str(i) for i in range(len(vertices_by_col["x"]))]

# Plot the edges

lines = {}
for start_vertex, adjacent in poly["adjacency"].items():
    for end_vertex in adjacent:
        line = frozenset((int(start_vertex), int(end_vertex)))
        count = lines.get(line, 0)
        lines[line] = count + 1
        
for line, count in lines.items():
    #print(line, count)
    assert count == 2, "Should be dupes"

lines_by_col = {dim: [] for dim in dims}
lines_by_col["line"] = []
for line in lines:
    for vertex in line:
        for dim in dims:
            lines_by_col[dim].append(vertices_by_col[dim][vertex])
        lines_by_col["line"].append(str(set(line)))

#(df=vertices_by_col, x="x", y="y", z="z", color="extreme", text="text")
fig = px.line_3d(lines_by_col, x="x", y="y", z="z", color="line")
#fig.add_scatter3d(**{dim: vertices_by_col[dim] for dim in dims})
fig.show()

In [12]:
with open("states.out") as states_file:
    states_data = json.load(states_file)

RSOut = namedtuple("RSOut", ["param", "decomp"])

states  = [RSOut(np.array(item["param"]["data"]), np.array(item["minkowski_decomp"])) for item in states_data]

for state in states:
    print(state)


RSOut(param=array([0.47752922, 0.67299757, 0.56483636]), decomp=array([6, 9, 5, 5, 7, 1, 3, 7, 6, 4]))
RSOut(param=array([0.7019203 , 0.61131388, 0.36551776]), decomp=array([6, 9, 6, 1, 7, 1, 3, 7, 6, 4]))
RSOut(param=array([0.82011963, 0.45792982, 0.34308028]), decomp=array([6, 9, 6, 1, 7, 1, 7, 7, 6, 4]))
RSOut(param=array([0.90948196, 0.26338513, 0.32166883]), decomp=array([6, 9, 6, 1, 7, 1, 7, 7, 2, 4]))
RSOut(param=array([0.91331607, 0.36943678, 0.17137743]), decomp=array([6, 9, 6, 1, 7, 1, 7, 7, 2, 2]))
RSOut(param=array([0.94933105, 0.17734095, 0.25946243]), decomp=array([6, 9, 6, 1, 7, 1, 7, 7, 2, 6]))
RSOut(param=array([0.91810112, 0.39337156, 0.04846801]), decomp=array([1, 9, 6, 1, 7, 1, 7, 7, 2, 6]))
RSOut(param=array([0.97086914, 0.1714741 , 0.16736115]), decomp=array([6, 9, 6, 1, 7, 1, 7, 3, 2, 6]))
RSOut(param=array([0.98765442, 0.05698219, 0.14591699]), decomp=array([4, 9, 6, 1, 7, 1, 7, 3, 2, 6]))
RSOut(param=array([0.93961575, 0.01293559, 0.34198672]), decomp=array([6,

In [57]:
np_poly = np.array(point_clouds)

for state in states:
    param = np.array(state.param)
    scores = np_poly @ param
    maximised = np.argmax(scores, axis=1)
    print("python: ", maximised)
    print("rs out: ", np.array(state.decomp))
    assert(np.array_equal(state.decomp, maximised))


python:  [6 9 5 5 7 1 3 7 6 4]
rs out:  [6 9 5 5 7 1 3 7 6 4]
python:  [6 9 6 1 7 1 3 7 6 4]
rs out:  [6 9 6 1 7 1 3 7 6 4]
python:  [6 9 6 1 7 1 7 7 6 4]
rs out:  [6 9 6 1 7 1 7 7 6 4]
python:  [6 9 6 1 7 1 7 7 2 4]
rs out:  [6 9 6 1 7 1 7 7 2 4]
python:  [6 9 6 1 7 1 7 7 2 2]
rs out:  [6 9 6 1 7 1 7 7 2 2]
python:  [6 9 6 1 7 1 7 7 2 6]
rs out:  [6 9 6 1 7 1 7 7 2 6]
python:  [1 9 6 1 7 1 7 7 2 6]
rs out:  [1 9 6 1 7 1 7 7 2 6]
python:  [6 9 6 1 7 1 7 3 2 6]
rs out:  [6 9 6 1 7 1 7 3 2 6]
python:  [4 9 6 1 7 1 7 3 2 6]
rs out:  [4 9 6 1 7 1 7 3 2 6]
python:  [6 9 6 1 7 1 7 6 2 6]
rs out:  [6 9 6 1 7 1 7 6 2 6]


Now we try to compute a Minkowski sum directly through qhull

- I'm giving up on using qhull as its numerical stability mechanisms mean that we get different results

In [58]:
def qhull(cloud):
    hull = ConvexHull(cloud)
    print(hull.simplices)
    return sorted({v for h_v in ConvexHull(cloud).vertices for v in h_v})


hulls = [qhull(cloud) for cloud in point_clouds]

MinkSumPoint = namedtuple("MinkSumPoint", ["point", "decomp"])


def mink_sum(point_clouds, hulls):
    hull_points = [
        [MinkSumPoint(cloud[vertex], [vertex]) for vertex in hull]
        for cloud, hull in zip(point_clouds, hulls)
    ]
    hull_points.reverse()
    mink_sum = hull_points.pop()
    for operand in hull_points:
        print(f"Input lens {len(mink_sum)}, {len(operand)}")
        point_product = [
            MinkSumPoint([x_a + x_b for x_a, x_b in zip(a.point, b.point)], a.decomp + b.decomp)
            for a, b in product(mink_sum, operand)
        ]
        mink_sum = [point_product[i] for i in qhull([point for point, _ in point_product])]
        print(f"Lengths {len(point_product)}, {len(mink_sum)}")
    return mink_sum

num_polys = 10
mink_sum = mink_sum(point_clouds[:num_polys], hulls[:num_polys])

for vertex in mink_sum:
    print(vertex.decomp)

[3-simplex in 3D space
Vertices:
	(1.0167892852172524, 1.1594778292564127, -0.339132028274736)
	(-0.3671771556690937, 1.0776083083046823, -0.4585935533768065)
	(-0.523728304823436, 0.5133766634286447, 0.4521352279285804), 3-simplex in 3D space
Vertices:
	(0.746160899162809, 1.5770927586151724, 1.29778838385861)
	(-0.3671771556690937, 1.0776083083046823, -0.4585935533768065)
	(1.0167892852172524, 1.1594778292564127, -0.339132028274736), 3-simplex in 3D space
Vertices:
	(0.5714356660369535, 0.7167595493845169, 0.320733465386696)
	(1.0167892852172524, 1.1594778292564127, -0.339132028274736)
	(-0.523728304823436, 0.5133766634286447, 0.4521352279285804), 3-simplex in 3D space
Vertices:
	(0.366339949442874, 0.30810747672287087, 1.2317268686174188)
	(0.5714356660369535, 0.7167595493845169, 0.320733465386696)
	(-0.523728304823436, 0.5133766634286447, 0.4521352279285804), 3-simplex in 3D space
Vertices:
	(0.5714356660369535, 0.7167595493845169, 0.320733465386696)
	(0.9672275386280861, 0.6595944